In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import tree,ensemble,metrics

from rule import Rule
from rule_extraction import rule_extract,draw_tree,rules_vote

import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from deap import creator, base, tools, algorithms
import sys
from datetime import datetime

C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:
def avg(l):
    """
    Returns the average between list elements
    """
    return (sum(l)/float(len(l)))


def getFitness(individual, X, y):
    """
    Feature subset fitness function
    """

    if(individual.count(0) != len(individual)):
        # get index with value 0
        cols = [index for index in range(
            len(individual)) if individual[index] == 0]

        # get features subset
        X_parsed = X.drop(X.columns[cols], axis=1)
        X_subset = pd.get_dummies(X_parsed)

        # apply classification algorithm
        clf = LogisticRegression()

        return (avg(cross_val_score(clf, X_subset, y, cv=5)),)
    else:
        return(0,)


def geneticAlgorithm(X, y, n_population, n_generation):
    """
    Deap global variables
    Initialize variables to use eaSimple
    """
    # create individual
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)

    # create toolbox
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat,
                     creator.Individual, toolbox.attr_bool, len(X.columns))
    toolbox.register("population", tools.initRepeat, list,
                     toolbox.individual)
    toolbox.register("evaluate", getFitness, X=X, y=y)
    toolbox.register("mate", tools.cxOnePoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)

    # initialize parameters
    pop = toolbox.population(n=n_population)
    hof = tools.HallOfFame(n_population * n_generation)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)

    # genetic algorithm
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2,
                                   ngen=n_generation, stats=stats, halloffame=hof,
                                   verbose=True)

    # return hall of fame
    return hof


def bestIndividual(hof, X, y):
    """
    Get the best individual
    """
    maxAccurcy = 0.0
    for individual in hof:
    	print(individual.fitness.values)
    	print(maxAccurcy)
    	if(individual.fitness.values[0] > maxAccurcy):
            maxAccurcy = individual.fitness.values
            _individual = individual

    _individualHeader = [list(X)[i] for i in range(
        len(_individual)) if _individual[i] == 1]
    return _individual.fitness.values, _individual, _individualHeader


# def getArguments():
#     """
#     Get argumments from command-line
#     If pass only dataframe path, pop and gen will be default
#     """
#     dfPath = sys.argv[1]
#     if(len(sys.argv) == 4):
#         pop = int(sys.argv[2])
#         gen = int(sys.argv[3])
#     else:
#         pop = 10
#         gen = 2
#     return dfPath, pop, gen


def gafeature():
    # get dataframe path, population number and generation number from command-line argument
    n_pop = 20
    n_gen = 6
    # read dataframe from csv
    df = pd.read_csv('heartfailure.csv')

    # encode labels column to numbers
    le = LabelEncoder()
    le.fit(df.iloc[:, -1])
    y = le.transform(df.iloc[:, -1])
    X = df.iloc[:, :-1]

    # get accuracy with all features
    individual = [1 for i in range(len(X.columns))]
    print("Accuracy with all features: \t" + str(getFitness(individual, X, y)) + "\n")

    # apply genetic algorithm
    hof = geneticAlgorithm(X, y, n_pop, n_gen)

    # select the best individual
    accuracy, individual, header = bestIndividual(hof, X, y)
    print('Best Accuracy: \t' + str(accuracy))
    print('Number of Features in Subset: \t' + str(individual.count(1)))
    print('Individual: \t\t' + str(individual))
    print('Feature Subset\t: ' + str(header))

    return header

In [3]:
gafeature()

C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solv

Accuracy with all features: 	(0.7331823182318231,)



C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solv

C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solv

C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solv

...

5  	15    	0.777929	0.767305	0.787547


C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solv

6  	18    	0.784429	0.771409	0.787547
(0.7875471547154715,)
0.0
(0.7875471547154715,)
(0.7875471547154715,)
(0.7853863386338634,)
(0.7875471547154715,)
(0.7853863386338634,)
(0.7875471547154715,)
(0.7815471547154715,)
(0.7875471547154715,)
(0.7795871587158716,)
(0.7875471547154715,)
(0.7775669566956696,)
(0.7875471547154715,)
(0.7732853285328534,)
(0.7875471547154715,)
(0.7714085408540854,)
(0.7875471547154715,)
(0.7714085408540854,)
(0.7875471547154715,)
(0.7714085408540854,)
(0.7875471547154715,)
(0.7712853285328534,)
(0.7875471547154715,)
(0.7692647264726473,)
(0.7875471547154715,)
(0.7692449244924492,)
(0.7875471547154715,)
(0.7673051305130513,)
(0.7875471547154715,)
(0.7514049404940495,)
(0.7875471547154715,)
(0.7476243624362436,)
(0.7875471547154715,)
(0.7472445244524453,)
(0.7875471547154715,)
(0.7472445244524453,)
(0.7875471547154715,)
(0.7470841084108412,)
(0.7875471547154715,)
(0.7452433243324335,)
(0.7875471547154715,)
(0.7415637563756377,)
(0.7875471547154715,)
(0.737041704

C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


['anaemia', 'creatinine_phosphokinase', 'serum_creatinine', 'sex', 'time']

In [4]:
# fetch dataset
data = pd.read_csv('heartfailure.csv', usecols = ['anaemia', 'serum_creatinine', 'serum_sodium', 'time','DEATH_EVENT','age'])

# drop NA records since most Tree algorithm cannot handle
data.dropna(inplace=True)

# split training/test sets
X_train, X_test, y_train, y_test = train_test_split(data[['anaemia', 'serum_creatinine', 'serum_sodium', 'time','DEATH_EVENT','age']], 
                                                    data.DEATH_EVENT, test_size=0.2, random_state=0)

# dataset shape
print(X_train.shape, X_test.shape)
print(X_train.head(5))

(399, 6) (100, 6)
     anaemia  serum_creatinine  serum_sodium  time  DEATH_EVENT   age
107        1               0.9           138    88            0  45.0
428        1               6.1           131   106            0  60.0
71         0               0.9           139    71            0  58.0
473        0               1.0           135   205            0  69.0
6          1               1.2           137    10            1  75.0


In [5]:
X_train_proceeded = X_train[['age','anaemia', 'serum_creatinine', 'serum_sodium', 'time']]
X_test_proceeded = X_test[['age','anaemia', 'serum_creatinine', 'serum_sodium', 'time']]
print(X_train_proceeded.head())

      age  anaemia  serum_creatinine  serum_sodium  time
107  45.0        1               0.9           138    88
428  60.0        1               6.1           131   106
71   58.0        0               0.9           139    71
473  69.0        0               1.0           135   205
6    75.0        1               1.2           137    10


In [6]:
model_RF_clf = ensemble.RandomForestClassifier(max_depth=4,n_estimators=3,random_state=9)
model_RF_clf.fit(X_train_proceeded,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=4, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=3,
                       n_jobs=None, oob_score=False, random_state=9, verbose=0,
                       warm_start=False)

In [7]:
# model performance on test set
y_pred_test = model_RF_clf.predict(X_test_proceeded)
print(metrics.confusion_matrix(y_test,y_pred_test))

[[65  2]
 [ 9 24]]


In [8]:
start_time = datetime.now()
rules,_ = rule_extract(model=model_RF_clf,feature_names=X_train_proceeded.columns)
for i in rules:
    print(i)
end_time = datetime.now()

time <= 73.5 and age > 59.0 and time > 31.5 and time > 72.5
time > 73.5 and serum_creatinine <= 1.1899999976158142 and time <= 78.5 and age <= 63.0
time <= 73.5 and age <= 59.0 and age > 51.5 and age > 53.5
time > 73.5 and serum_creatinine > 1.1899999976158142 and time > 173.0 and serum_sodium <= 129.5
time <= 73.5 and age <= 59.0 and age > 51.5 and age <= 53.5
time <= 73.5 and age > 59.0 and time <= 31.5
time > 73.5 and serum_creatinine > 1.1899999976158142 and time <= 173.0 and serum_creatinine > 1.6500000357627869
time > 73.5 and serum_creatinine <= 1.1899999976158142 and time <= 78.5 and age > 63.0
time <= 73.5 and age <= 59.0 and age <= 51.5 and serum_sodium > 136.5
time > 73.5 and serum_creatinine > 1.1899999976158142 and time <= 173.0 and serum_creatinine <= 1.6500000357627869
time <= 73.5 and age > 59.0 and time > 31.5 and time <= 72.5
time > 73.5 and serum_creatinine > 1.1899999976158142 and time > 173.0 and serum_sodium > 129.5
time <= 73.5 and age <= 59.0 and age <= 51.5 and

In [9]:
print('Duration: {}'.format(end_time - start_time))

Duration: 0:00:00.013964
